# Flight Path

Extracting the flight path coordinates from the telemetry and creating a KML file.

In [2]:
import numpy as np
import pandas as pd

directory = '../data'
mission_data = pd.read_csv(f'{directory}/clean-data.csv', na_values="*", index_col=0)
mission_data.head()

,millis,year,month,day,hour,minute,second,voltage,satellites,hdop,...,latitude,longitude,temperature_C,pressure_Pa,pressure_altitude_m,humidity_RH,mission_millis,blended_altitude_m,altitude_m,ascent_rate_mps
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-09-25 17:28:11,135015,2021.0,9.0,25.0,17.0,28.0,11.0,4.40,6.0,15.71,...,47.160885,-120.848351,20.34,93890.44,638.23,39.34,0,640.489791,657.3,NaN
2021-09-25 17:28:12,136017,2021.0,9.0,25.0,17.0,28.0,12.0,4.37,7.0,2.05,...,47.160892,-120.848351,20.39,93892.72,638.02,39.21,1002,640.287907,660.1,2.800000
2021-09-25 17:28:13,137021,2021.0,9.0,25.0,17.0,28.0,13.0,4.37,7.0,2.06,...,47.160877,-120.848343,20.45,93891.80,638.10,39.26,2006,640.369369,662.6,2.645000
2021-09-25 17:28:14,138027,2021.0,9.0,25.0,17.0,28.0,14.0,4.37,8.0,2.05,...,47.160824,-120.848351,20.49,93893.68,637.94,39.78,3012,640.202904,663.1,1.881822
2021-09-25 17:28:15,139032,2021.0,9.0,25.0,17.0,28.0,15.0,4.43,8.0,1.86,...,47.160759,-120.848343,20.53,93889.08,638.34,40.20,4017,640.610215,661.0,0.784685


In [3]:
burst_index = np.argmax(mission_data['gps_altitude_m'])
burst_time = mission_data.index[burst_index]

ascent_phase = mission_data[mission_data.index < burst_time]
descent_phase = mission_data[mission_data.index > burst_time]

ascent_rate = ascent_phase['ascent_rate_mps']
descent_rate = descent_phase['ascent_rate_mps'] * -1

burst_row = mission_data.iloc[burst_index]
burst_altitude = burst_row['altitude_m']

print(burst_altitude)

33502.8


In [4]:
import simplekml
kml = simplekml.Kml()
kml.document.name = "Graupel-2 Actual Flight Path"

def create_point(row, name):
    point = kml.newpoint(name=name)
    latitude = row['latitude']
    longitude = row['longitude']
    altitude = row['altitude_m']
    time = row.name
    coords = row[['longitude','latitude','altitude_m']].to_numpy()
    coords.resize(1,3)
    point.coords = coords
    point.description = f'{name} at {latitude:0.4f}, {longitude:0.4f}, {altitude:0.0f}m at {time}'
    point.altitudemode = simplekml.AltitudeMode.absolute
    
def summary_stats(series):
    return f'{series.mean():0.1f}m/s ({series.max():0.1f} max)'

create_point(mission_data.iloc[0], 'Launch')
create_point(burst_row, 'Burst')
create_point(mission_data.iloc[-1], 'Landing')

every_minute_data = pd.concat([mission_data.iloc[::60], mission_data.iloc[[-1]]])

path = kml.newlinestring(name='Flight path')
path.description = f'Ascent rate: {summary_stats(ascent_rate)}, descent rate: {summary_stats(descent_rate)}'
path.coords = every_minute_data[['longitude','latitude','altitude_m']].to_numpy()
path.extrude = 1
path.altitudemode = simplekml.AltitudeMode.absolute
path.style.linestyle.color = "ffff0018"
path.style.linestyle.width = 4
path.style.polystyle.color = "ffff0018"

kml.save(f'{directory}/recorded-flight-path.kml')

In [5]:
print(kml.kml(format=True))

<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">
    <Document id="1">
        <Style id="10">
            <LineStyle id="11">
                <color>ffff0018</color>
                <colorMode>normal</colorMode>
                <width>4</width>
            </LineStyle>
            <PolyStyle id="12">
                <color>ffff0018</color>
                <colorMode>normal</colorMode>
                <fill>1</fill>
                <outline>1</outline>
            </PolyStyle>
        </Style>
        <name>Graupel-2 Actual Flight Path</name>
        <Placemark id="3">
            <name>Launch</name>
            <description>Launch at 47.1609, -120.8484, 657m at 2021-09-25 17:28:11</description>
            <Point id="2">
                <coordinates>-120.84835052,47.16088485,657.3</coordinates>
                <altitudeMode>absolute</altitudeMode>
            </Point>
        </Placemark>
        <Placemar